# Moving all the 'good' LEAP stores to the public bucket

We should log these somewhere and then enable ingestion later (just like with the legacy stores)

In [1]:
from leap_data_management_utils.cmip_utils import CMIPBQInterface, IIDEntry
from leap_data_management_utils.cmip_testing import test_all

import os
import json
import gcsfs
import zarr
from tqdm.auto import tqdm

In [2]:
table_id = "leap-pangeo.testcmip6.cmip6_feedstock_test2"
bq = CMIPBQInterface(table_id)
df_legacy = bq.get_latest()

# log all moves in a separate bigquery table
table_id = "leap-pangeo.cmip6_pgf_ingestion.leap_legacy"
bq_legacy = CMIPBQInterface(table_id)

In [3]:
# test that I can actually copy things from in here (cant, need to authenticate):
# This might disspear (its in some 1Password secured folder)
# TODO refactor this to the hub?
key_file = "/private/var/folders/m2/vwrjf3zx1bvdbhfkw7bxpz7r0000gn/T/com.agilebits.onepassword7/com.agilebits.Attachments.noindex/ipigjxgmqrevlaplfwomu4sva4/leap-pangeo-93546a72d2da.json"

with open(key_file) as f:
    token = json.load(f)
    fs = gcsfs.GCSFileSystem(token=token)

In [5]:
for idx, row in tqdm(df_legacy.iterrows(), total=len(df_legacy)):
    iid = row.instance_id
    source = "gs://" + row.store
    target_prefix = "gs://cmip6/CMIP6_LEAP_legacy"
    target = os.path.join(target_prefix, *source.split("/")[-2:])
    # check if iid is already in the target bq
    if not bq_legacy.iid_exists(iid):
        print(f"Testing {iid=}")
        try:
            test_all(store=zarr.storage.FSStore(source, fs=fs), iid=iid, verbose=False)
            tests_passed = True
            print("Test PASSED ✅")
        except:
            tests_passed = False
            print("Test FAILED ❌")
        iid_entry = IIDEntry(
            iid=iid, store=target, retracted=False, tests_passed=tests_passed
        )
        # print(f"Copy {iid=} \nfrom \n{source=} \nto \n{target=}")
        fs.cp(source, target, recursive=True)
        bq_legacy.insert_iid(iid_entry)

  0%|          | 0/3411 [00:00<?, ?it/s]


KeyboardInterrupt

